# create pipelines and aggregate function

In [8]:
import pymongo
from datetime import datetime
import json

# insert data 

In [3]:
client=pymongo.MongoClient('localhost',27017)


In [179]:
# load data to database
db=client['library']
collection=db['authors']
collection.delete_many({})
with open('data/authors.json','r') as f:
    file=json.load(f)
collection.insert_many(file)



InsertManyResult([131, 134, 136, 137, 138, 139, 140, 141, 142, 143, 144, 146, 147, 149, 150, 152, 153, 154, 155, 156, 157, 159, 160, 161, 162, 163, 164, 165, 166, 168, 170, 171, 175, 179, 180, 184, 185, 188, 189, 191, 192, 194, 195, 196, 199, 200, 100, 101, 102, 104, 105, 106, 107, 110, 112, 113, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127], acknowledged=True)

In [14]:
collection=db['users']

with open('data/users.json','r') as f:
    file=json.load(f)
collection.insert_many(file)

collection=db['books']

with open('data/books.json','r') as f:
    file=json.load(f)
collection.insert_many(file)

InsertManyResult([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100], acknowledged=True)

# regex

In [124]:
data=collection.find({'company.phone':{'$regex':'^\+1 \(\d{3}\) \d{3}-78\d{2}$'}})
for a in data:
    print(a)

{'_id': ObjectId('67349ae5fb1a3fd2db1dca50'), 'index': 1, 'name': 'Person 1', 'isActive': False, 'registered': '2023-02-04 10:30:00', 'age': 21, 'gender': 'female', 'eyeColor': 'green', 'favoriteFruit': 'banana', 'company': {'title': 'Company 1', 'email': 'person1@company1.com', 'phone': '+1 (123) 457-7891', 'location': {'country': 'USA', 'address': '100 Main Street'}}, 'tags': ['tag1', 'tag2', 'tag3']}
{'_id': ObjectId('67349ae5fb1a3fd2db1dca51'), 'index': 2, 'name': 'Person 2', 'isActive': True, 'registered': '2023-03-07 10:30:00', 'age': 22, 'gender': 'male', 'eyeColor': 'brown', 'favoriteFruit': 'orange', 'company': {'title': 'Company 2', 'email': 'person2@company2.com', 'phone': '+1 (123) 458-7892', 'location': {'country': 'USA', 'address': '200 Main Street'}}, 'tags': ['tag2', 'tag3', 'tag4']}
{'_id': ObjectId('67349ae5fb1a3fd2db1dca52'), 'index': 3, 'name': 'Person 3', 'isActive': False, 'registered': '2023-04-10 10:30:00', 'age': 23, 'gender': 'female', 'eyeColor': 'hazel', 'fa

# pipeline
[
    
    {}, stage 1 source
    
    {}, stage 2 
]

In [32]:
# how many uses are active
collection=db['users']
pipeline=[
    {'$match':{'gender':'female'}},
    {'$group':{'_id':'$isActive','count isActive':{'$sum':1}}}
]

result=collection.aggregate(pipeline)
for data in result:
    print(data)

{'_id': False, 'count isActive': 50}


In [ ]:
#what is the average age of all users/gender
collection=db['users']
pipeline=[
    {'$group':{'_id':'$gender','averageAge':{'$avg':'$age'}}}

]
result=collection.aggregate(pipeline)
for r in result:
    print(r)

{'_id': 'female', 'averageAge': 70.0}
{'_id': 'male', 'averageAge': 71.0}


In [ ]:
# list the top 2 common favorute fruits among users
pipeline=[
    {'$group':{'_id':'$favoriteFruit','count':{'$sum':1}}},
    {'$sort':{'count':-1}},
    {'$limit':2}
]
result=collection.aggregate(pipeline)
for r in result:
    print(r)

{'_id': 'banana', 'count': 20}
{'_id': 'pineapple', 'count': 20}


In [57]:
# find the total number of males and females 
pipeline=[
    {'$group':{'_id':'$gender','count':{'$sum':1}}}
]
result=collection.aggregate(pipeline)
for r in result:
    print(r)

{'_id': 'female', 'count': 50}
{'_id': 'male', 'count': 50}


In [68]:
# which country has the highest number of registered user 
pipeline=[
    {'$group':{'_id':'$company.location.country','user_count':{'$sum':1}}},
    {'$sort':{'user_count':-1}},
    {'$limit':1}
]
result=collection.aggregate(pipeline)
for r in result:
    print(r)

{'_id': 'USA', 'user_count': 100}


In [114]:
# list the unique eyecolor in the collection
pipeline=[
    {'$unique':'$eyeColor'}
]
result=collection.distinct('eyeColor')
for r in result:
    print(r,'\n')

# using aggregate function
pipeline=[
    {'$group':{'_id':'$eyeColor'}},
    # {'$project':{'_id':0,'city':'$_id'}}
]
result=collection.distinct('eyeColor')
for r in result:
    print(r)

blue 

brown 

gray 

green 

hazel 

blue
brown
gray
green
hazel


# pipelines for array

In [92]:
# what is the average number of tags per person
pipeline=[
    {'$unwind':{'path':'$tags'}},
    {'$group':{'_id':'$_id','Avg_Tag':{'$sum':1}}},
    {'$group':{'_id':None,'average_tag_count':{'$avg':'$Avg_Tag'}}}
]
result=collection.aggregate(pipeline)
for r in result:
    print(r)

{'_id': None, 'average_tag_count': 3.0}


In [95]:
# method 2 using $size
pipeline=[
    {'$addFields':{'numberofTags':{
        '$size':{'$ifNull':['$tags',[]]}
    }}},
    {'$group':{'_id':None,'avg_no_tags':{'$avg':'$numberofTags'}}}
]
result=collection.aggregate(pipeline)
for r in result:
    print(r)

{'_id': None, 'avg_no_tags': 3.0}


# match and project pipeline

In [103]:
# how many users have tag20 as their tags

pipeline=[
    {'$match':{'tags':'tag20'}},
    {'$count':'userwithTag20'}
]
result=collection.aggregate(pipeline)
for r in result:
    print(r)

{'userwithTag20': 3}


In [ ]:
# what are the names and age of uses who are inactive and have tag20 as a tag
pipeline=[
    {'$match':{'tags':'tag9','isActive':False}},
    {'$project':{'name':'$name','age':'$age'}}
]
result=collection.aggregate(pipeline)
for r in result:
    print(r)

{'_id': ObjectId('67349ae5fb1a3fd2db1dca56'), 'name': 'Person 7', 'age': 27}
{'_id': ObjectId('67349ae5fb1a3fd2db1dca58'), 'name': 'Person 9', 'age': 29}


In [113]:
#using or operator in match 
pipeline=[
    {'$match':{
        '$or':[
            {'tags':'tag10'},{'tags':'tag20'}
        ]
    }}
]
result=collection.aggregate(pipeline)
for r in result:
    print(r)

{'_id': ObjectId('67349ae5fb1a3fd2db1dca57'), 'index': 8, 'name': 'Person 8', 'isActive': True, 'registered': '2023-09-25 10:30:00', 'age': 28, 'gender': 'male', 'eyeColor': 'hazel', 'favoriteFruit': 'grape', 'company': {'title': 'Company 8', 'email': 'person8@company8.com', 'phone': '+1 (123) 464-7898', 'location': {'country': 'USA', 'address': '800 Main Street'}}, 'tags': ['tag8', 'tag9', 'tag10']}
{'_id': ObjectId('67349ae5fb1a3fd2db1dca58'), 'index': 9, 'name': 'Person 9', 'isActive': False, 'registered': '2023-10-28 10:30:00', 'age': 29, 'gender': 'female', 'eyeColor': 'gray', 'favoriteFruit': 'pineapple', 'company': {'title': 'Company 9', 'email': 'person9@company9.com', 'phone': '+1 (123) 465-7899', 'location': {'country': 'USA', 'address': '900 Main Street'}}, 'tags': ['tag9', 'tag10', 'tag11']}
{'_id': ObjectId('67349ae5fb1a3fd2db1dca59'), 'index': 10, 'name': 'Person 10', 'isActive': True, 'registered': '2023-11-03 10:30:00', 'age': 30, 'gender': 'male', 'eyeColor': 'blue', '

In [ ]:
# how many users have a phone ending with -78**
pipeline=[
    {'$match':{'company.phone':{'$regex':'^\+1 \(\d{3}\) \d{3}-78\d{2}$'}}},
    {'$group':{'_id':None,'count':{'$sum':1}}}
]
result=collection.aggregate(pipeline)
for r in result:
    print(r)

{'_id': None, 'count': 9}


In [136]:
#who has registeded the most recently 
pipeline=[
    {'$sort':{'registered':-1}},
    {'$limit':4},
    {'$project':{'name':'$name','registered':'$registered'}}
]
result=collection.aggregate(pipeline)
for r in result:
    print(r)

{'_id': ObjectId('67349ae5fb1a3fd2db1dcaa2'), 'name': 'Person 83', 'registered': '2023-12-26 10:30:00'}
{'_id': ObjectId('67349ae5fb1a3fd2db1dca72'), 'name': 'Person 35', 'registered': '2023-12-22 10:30:00'}
{'_id': ObjectId('67349ae5fb1a3fd2db1dca96'), 'name': 'Person 71', 'registered': '2023-12-18 10:30:00'}
{'_id': ObjectId('67349ae5fb1a3fd2db1dca66'), 'name': 'Person 23', 'registered': '2023-12-14 10:30:00'}


In [150]:
#categorize users by their favourite fruits
pipeline=[
    {'$group':{'_id':'$favoriteFruit',
    'users':{'$push':'$name'}}}
]
result=collection.aggregate(pipeline)
for r in result:
    print(r)

{'_id': 'orange', 'users': ['Person 2', 'Person 7', 'Person 12', 'Person 17', 'Person 22', 'Person 27', 'Person 32', 'Person 37', 'Person 42', 'Person 47', 'Person 52', 'Person 57', 'Person 62', 'Person 67', 'Person 72', 'Person 77', 'Person 82', 'Person 87', 'Person 92', 'Person 97']}
{'_id': 'banana', 'users': ['Person 1', 'Person 6', 'Person 11', 'Person 16', 'Person 21', 'Person 26', 'Person 31', 'Person 36', 'Person 41', 'Person 46', 'Person 51', 'Person 56', 'Person 61', 'Person 66', 'Person 71', 'Person 76', 'Person 81', 'Person 86', 'Person 91', 'Person 96']}
{'_id': 'pineapple', 'users': ['Person 4', 'Person 9', 'Person 14', 'Person 19', 'Person 24', 'Person 29', 'Person 34', 'Person 39', 'Person 44', 'Person 49', 'Person 54', 'Person 59', 'Person 64', 'Person 69', 'Person 74', 'Person 79', 'Person 84', 'Person 89', 'Person 94', 'Person 99']}
{'_id': 'apple', 'users': ['Person 5', 'Person 10', 'Person 15', 'Person 20', 'Person 25', 'Person 30', 'Person 35', 'Person 40', 'Perso

In [163]:
#how many users have tag10 as their 2nd tag in the list of tags
pipeline=[
    {'$match':{'tags.1':'tag10'}},
    # {'$count':'matching tags at pos 2'}
    
]
result=collection.aggregate(pipeline)
for r in result:
    print(r)

{'_id': ObjectId('67349ae5fb1a3fd2db1dca58'), 'index': 9, 'name': 'Person 9', 'isActive': False, 'registered': '2023-10-28 10:30:00', 'age': 29, 'gender': 'female', 'eyeColor': 'gray', 'favoriteFruit': 'pineapple', 'company': {'title': 'Company 9', 'email': 'person9@company9.com', 'phone': '+1 (123) 465-7899', 'location': {'country': 'USA', 'address': '900 Main Street'}}, 'tags': ['tag9', 'tag10', 'tag11']}


In [ ]:
# find users who have both tag10 and tag 11 as their tags 
pipeline=[
    {'$match':{'$or':[{'tags':'tag10'},{'tags':'tag11'}]}}
]
result=collection.aggregate(pipeline)
# for r in result:
#     print(r)

# alternative 
pipeline=[
    {'$match':{'tags':{'$all':['tag10','tag11']}}}
]
result=collection.aggregate(pipeline)
for r in result:
    print(r)

{'_id': ObjectId('67349ae5fb1a3fd2db1dca58'), 'index': 9, 'name': 'Person 9', 'isActive': False, 'registered': '2023-10-28 10:30:00', 'age': 29, 'gender': 'female', 'eyeColor': 'gray', 'favoriteFruit': 'pineapple', 'company': {'title': 'Company 9', 'email': 'person9@company9.com', 'phone': '+1 (123) 465-7899', 'location': {'country': 'USA', 'address': '900 Main Street'}}, 'tags': ['tag9', 'tag10', 'tag11']}
{'_id': ObjectId('67349ae5fb1a3fd2db1dca59'), 'index': 10, 'name': 'Person 10', 'isActive': True, 'registered': '2023-11-03 10:30:00', 'age': 30, 'gender': 'male', 'eyeColor': 'blue', 'favoriteFruit': 'apple', 'company': {'title': 'Company 10', 'email': 'person10@company10.com', 'phone': '+1 (123) 466-7900', 'location': {'country': 'USA', 'address': '1000 Main Street'}}, 'tags': ['tag10', 'tag11', 'tag12']}


In [177]:
#list all companies located in the USA with their corresponding user counts
pipeline=[
    {'$match':{'company.location.country':'USA'}},
    {'$group':{'_id':'$company.title','users count':{'$sum':+1}}}
]
result=collection.aggregate(pipeline)
for r in result:
    print(r)

{'_id': 'Company 50', 'users count': 1}
{'_id': 'Company 42', 'users count': 1}
{'_id': 'Company 8', 'users count': 1}
{'_id': 'Company 19', 'users count': 1}
{'_id': 'Company 47', 'users count': 1}
{'_id': 'Company 7', 'users count': 1}
{'_id': 'Company 73', 'users count': 1}
{'_id': 'Company 45', 'users count': 1}
{'_id': 'Company 80', 'users count': 1}
{'_id': 'Company 15', 'users count': 1}
{'_id': 'Company 96', 'users count': 1}
{'_id': 'Company 64', 'users count': 1}
{'_id': 'Company 12', 'users count': 1}
{'_id': 'Company 26', 'users count': 1}
{'_id': 'Company 6', 'users count': 1}
{'_id': 'Company 57', 'users count': 1}
{'_id': 'Company 2', 'users count': 1}
{'_id': 'Company 59', 'users count': 1}
{'_id': 'Company 75', 'users count': 1}
{'_id': 'Company 77', 'users count': 1}
{'_id': 'Company 87', 'users count': 1}
{'_id': 'Company 94', 'users count': 1}
{'_id': 'Company 98', 'users count': 1}
{'_id': 'Company 99', 'users count': 1}
{'_id': 'Company 39', 'users count': 1}
{'_i

# lookups

In [185]:
pipeline=[
    {
        '$lookup':{
            'from':'authors',
            'localField':'author_id',
            'foreignField':'_id',
            'as':'author_details'
        }
    },
    {
        '$addFields':{'author_details':{'$arrayElemAt':['$author_details',0]}}
    }
]

collection=db['books']
result=collection.aggregate(pipeline)
for r in result:
    print(r)


{'_id': 1, 'title': 'Performance indicate', 'author_id': 126, 'genre': 'Mystery', 'author_details': {'_id': 126, 'name': 'Alyssa Vazquez', 'birth_year': 1825}}
{'_id': 2, 'title': 'My if less thing', 'author_id': 100, 'genre': 'Mystery', 'author_details': {'_id': 100, 'name': 'William Randall', 'birth_year': 1719}}
{'_id': 3, 'title': 'Tax situation what hair', 'author_id': 171, 'genre': 'Classic', 'author_details': {'_id': 171, 'name': 'Patricia Parks', 'birth_year': 1747}}
{'_id': 4, 'title': 'Animal around product', 'author_id': 161, 'genre': 'Thriller', 'author_details': {'_id': 161, 'name': 'Brandy Martin', 'birth_year': 1912}}
{'_id': 5, 'title': 'Leg amount', 'author_id': 127, 'genre': 'Thriller', 'author_details': {'_id': 127, 'name': 'Rickey Miller', 'birth_year': 1741}}
{'_id': 6, 'title': 'Attention north', 'author_id': 192, 'genre': 'Dystopian', 'author_details': {'_id': 192, 'name': 'Kimberly Hernandez', 'birth_year': 1985}}
{'_id': 7, 'title': 'Doctor', 'author_id': 165, 